# Tutorial: Sending PySpark DataFrame to Arize

In the current version of Arize Python SDK, only Pandas DataFrames are supported. To log Spark DataFrames, which have `rdds` as their underlying structure, we will use `mapPartitions` to log them to arize.

# Install Dependencies in Colab

In [ ]:
!pip install -q pyspark
!pip install -q arize

# Parallelizing PySpark DataFrame
We first create a dummy PySpark DataFrame to send.


In [ ]:
import pyspark
from pyspark.sql import Row, SparkSession
import pandas as pd

spark = SparkSession.builder.getOrCreate()

# Read some dummy data for logging to Arize later
data = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/fixtures/compare-model-a.csv"
)
df_pandas = data[["loan_amount", "interest_rate", "grade", "prediction", "score"]]

df_pandas = pd.concat([df_pandas] * 5)

print("This is a pandas DataFrame:")
display(df_pandas)

# Create PySpark dataframe unparallelized
df_spark = spark.createDataFrame(df_pandas)

print("\nThis is the corresponding spark DataFrame")
df_spark.printSchema()

# Using `mapPartitions` to log each partition to Arize
`map_func` will be applied to each partition on of spark_df, allowing local copies of `pd.DataFrame` to be made and logged to Arize.

`success` will be returned if all entries in a particular partition has been logged properly. Otherwise, it will return the error code and error message for that partition instead.

`map_send_arize` should send `spark_df` to Arize with at least one of: `shap, prediction_labels, actual_labels`

## How To Log to Arize:
`your_spark_df.rdd.mapPartitions(map_func).collect()`

You will also need to update the `API_KEY` and `SPACE_KEY`
### Setting up Arize Client:
First, copy the Arize `API_KEY` and `SPACE_KEY` from your admin page linked below!




<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-keys.png" width="700">

In [ ]:
from arize.pandas.logger import Client, Schema
from arize.utils.types import ModelTypes, Environments

SPACE_KEY = "SPACE_KEY"
API_KEY = "API_KEY"
arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")

Define the function used by `pyspark.RDD.mapPartitions`

In [ ]:
import itertools
import uuid


def map_send_arize(pyspark_df_partition: itertools.chain):
    """
    Mapping function to be used to log to Arize
    """
    # Step 1: iterating through each Row to create pd.DataFrame
    pandas_df = None
    for row in pyspark_df_partition:
        row_dict = row.asDict()
        row_dict["prediction_id"] = str(uuid.uuid4())
        if pandas_df is None:
            pandas_df = pd.DataFrame(columns=row_dict.keys())
        pandas_df.loc[len(pandas_df)] = row_dict

    # Step 2: Define a Schema() object for Arize to pick up data from the correct columns for logging
    pandas_df_schema = Schema(
        prediction_id_column_name="prediction_id",  # REQUIRED
        prediction_label_column_name="prediction",
        prediction_score_column_name="score",
        feature_column_names=["loan_amount", "interest_rate", "grade"],
    )

    # Step 3: Log to arize
    response = arize_client.log(
        dataframe=pandas_df,
        schema=pandas_df_schema,
        model_id="pyspark-loan-model",
        model_version="1.0",
        model_type=ModelTypes.SCORE_CATEGORICAL,
        environment=Environments.PRODUCTION,
    )

    # Step 4: Check for errors when logging
    return iter([f"Status {response.status_code}: {response.text}"])

# Logging Example
For each `log` response we get from Arize, `collect()` will combine the iterables as the map function completes.

In [ ]:
%%time
log_output = df_spark.rdd.mapPartitions(map_send_arize).collect()
print(f"number of records sent: {len(df_pandas)}")
print(f"number of arize responses: {len(log_output)}")
if all(response.startswith("Status 200:") for response in log_output):
    print(f"✅ all log requests are successful")
else:
    for response in log_output:
        if not response.startswith("Status 200:"):
            print(f"❌ {response}")

# **Overview**


Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

### Website
Visit Us At: https://arize.com/model-monitoring/

### Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
- [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/)
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
- [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/)
- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.